In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
gdrive	sample_data


In [1]:
import os

home_dir = '/content/gdrive/My Drive/ATFTransformer'
home_dir = '/hy-tmp'
os.chdir(home_dir)
!pwd

!pip install tqdm
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

import multiprocessing
import time

from multiprocessing import Pool, freeze_support

/hy-tmp
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 217.3 kB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
data_dir = os.path.join(home_dir)
os.chdir(data_dir)
!pwd
# !ls

files = glob(os.path.join(data_dir, '*.npz'))

print(files)

/hy-tmp
['/hy-tmp/corn_histogram_all_full.npz']


In [5]:
filename = '/hy-tmp/corn_histogram_all_full.npz'

hists = np.load(filename)

print(type(hists), list(hists.keys()))
print(type(hists['output_image']), hists['output_image'].shape)

<class 'numpy.lib.npyio.NpzFile'> ['output_image', 'output_yield', 'output_year', 'output_locations', 'output_index', 'output_areas']
<class 'numpy.ndarray'> (320, 9, 32, 512)


In [15]:
freeze_support()

print(type(hists['output_image']), hists['output_image'].shape)

bins_name = list()
bins_valu = list()

for bin in range(0, hists['output_image'].shape[3]):
  bins_name.append(f'bin{bin}')

# print(bins_name)
print(hists['output_image'][0][0][0].shape)
# print(list(hists['output_image'][0][0][0]))
print(hists['output_index'][100][1])

# fn

idx_s = [x for x in range(0,319,29)]
idx_e = [x for x in range(29,320,29)]

print(len(idx_s), idx_s)
print(len(idx_e), idx_e)

# idx_s = [x for x in range(0,320,40)]
# idx_e = [x for x in range(40,321,40)]

# print(len(idx_s), idx_s)
# print(len(idx_e), idx_e)

# fn

def get_ds(idx_s, idx_e):
    hists = np.load('/hy-tmp/corn_histogram_all_full.npz')
    df = pd.DataFrame()

    time_idx = 0
    index_idx = 0
    for idx in range(idx_s, idx_e):
      for time_idx in range(0, hists['output_image'].shape[2]):
        for band_idx in range(0, hists['output_image'].shape[1]):
          bins_dict = dict(zip(bins_name,list(hists['output_image'][idx][band_idx][time_idx])))

          df1 = pd.DataFrame( hists['output_year'][idx], columns=['years'], index=[index_idx] )
          df2 = pd.DataFrame( hists['output_yield'][idx], columns=['yield'], index=[index_idx] )
          df3 = pd.DataFrame( hists['output_index'][idx][1], columns=['county'], index=[index_idx] )
          df4 = pd.DataFrame( f'band_{band_idx}', columns=['bands'], index=[index_idx] )
          df5 = pd.DataFrame( time_idx, columns=['time_idx'], index=[index_idx] )
          df6 = pd.DataFrame(bins_dict, index=[index_idx])

          df_r = pd.concat( [df1 , df2], axis=1)
          df_r = pd.concat( [df_r, df3], axis=1)
          df_r = pd.concat( [df_r, df4], axis=1)
          df_r = pd.concat( [df_r, df5], axis=1)
          df_r = pd.concat( [df_r, df6], axis=1)

          df = pd.concat( [df, df_r], axis=0)

          index_idx = index_idx + 1
            
    return df

with Pool(processes=11) as pool:
    results = pool.starmap(get_ds, zip(idx_s, idx_e))

# df.to_csv('china_prl.csv')


# fn
# for indx in range(0, hists.shape[0]):
#   for time_idx in range(0, hists.shape[2]):
#     for bands in range(0, hists.shape[1]):


<class 'numpy.ndarray'> (320, 9, 32, 512)
(512,)
9
11 [0, 29, 58, 87, 116, 145, 174, 203, 232, 261, 290]
11 [29, 58, 87, 116, 145, 174, 203, 232, 261, 290, 319]


NameError: name 'df' is not defined

In [25]:
print(type(results), len(results))

filesave = 'corn_china_pandas.csv'

df = pd.DataFrame()
for ii in range(0, 11):
    df = pd.concat( [df, results[ii]], axis=0 )
    
df.head()
    
df.to_csv(filesave)
    

<class 'list'> 11


In [26]:
ds = pd.read_csv(filesave)
ds[8340:8390]

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
8340,8340,2015,0.499838,1,band_6,30,0.000028,0.000197,0.000028,0.000085,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8341,8341,2015,0.499838,1,band_7,30,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8342,8342,2015,0.499838,1,band_8,30,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8343,8343,2015,0.499838,1,band_0,31,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8344,8344,2015,0.499838,1,band_1,31,0.000028,0.000084,0.000028,0.000056,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8345,8345,2015,0.499838,1,band_2,31,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8346,8346,2015,0.499838,1,band_3,31,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8347,8347,2015,0.499838,1,band_4,31,0.000141,0.000141,0.000028,0.000084,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8348,8348,2015,0.499838,1,band_5,31,0.000000,0.000028,0.000028,0.000028,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8349,8349,2015,0.499838,1,band_6,31,0.000113,0.000141,0.000282,0.000113,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
ds

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
0,0,2003,0.612950,0,band_0,0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2003,0.612950,0,band_1,0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2003,0.612950,0,band_2,0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2003,0.612950,0,band_3,0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2003,0.612950,0,band_4,0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91867,8347,2017,0.701433,29,band_4,31,0.000099,0.000049,0.000082,0.000016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91868,8348,2017,0.701433,29,band_5,31,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91869,8349,2017,0.701433,29,band_6,31,0.000049,0.000016,0.000082,0.000049,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91870,8350,2017,0.701433,29,band_7,31,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# !zip corn_china_pandas_prl11.csv.zip corn_china_pandas.csv
!cp corn_china_pandas.csv corn_china_pandas_prl11.csv.zip
!oss cp corn_china_pandas_prl11.csv.zip oss://

Start at 2022-12-12 08:56:43.514888525 +0000 UTC


Parallel:      5                   Jobs:          5                   
Threshold:     50.00MB             PartSize:      auto                
VerifyLength:  false               VerifyMd5:     false               
CheckpointDir: /root/.hycloud_ossutil_checkpoint

[-----------------------------------] 100.00% 6.44MB/s 192.06MB/192.06MB 30.01s
Waiting for the uploaded key to be completed on server side.


Upload successfully, 192.06MB, n/a, /hy-tmp/corn_china_pandas_prl11.csv.zip --> oss://corn_china_pandas_prl11.csv.zip, cost [31085], status [200], request id [00000185058D73109013CC4CA7715115]


In [ ]:
os.system("shutdown /s /t 1")

In [ ]:
print(type(hists['output_image']), hists['output_image'].shape)
print(type(hists['output_yield']), hists['output_yield'].shape)
print(type(hists['output_year']), hists['output_year'].shape)
print(type(hists['output_locations']), hists['output_locations'].shape)
print(type(hists['output_index']), hists['output_index'].shape)
print(hists['output_year'])
# print(hists['output_index'])
# fn

In [ ]:
from matplotlib import pyplot as plt
# print(hists['output_index'])

bands = [x for x in range(0,9,1)]
years = [x for x in range(2003, 2019, 1)]
times = [x for x in range(0,32,1)]

print('years :', years)
      
counties = [0, 1, 2, 3, 5, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 24, 26, 29]

num_bins = 512
bin_seq = np.linspace(1, num_bins, num_bins + 1) 

# fig, axs = plt.subplots(1, 9, figsize=(25,5))

it = 0
for county in range(0,len(counties),1):
  fig, axs = plt.subplots(1, 9, figsize=(25,5))
  for year in [0,1]:  #range(0,len(years),1): 
    for band in bands:
      for time in times:
        density = hists['output_image'][it,band,time,:]
        # print(len(density))
        axs[band].plot(bin_seq[0:-1], density)
      it = it + 1
    print(it)

  plt.show()
  